In [11]:
import tensorflow as tf 
import numpy as np

In [101]:
def generator(z, Output_dimension, n_units, reuse, alpha):
    with tf.variable_scope('generator', reuse=reuse):
        H_layer1=tf.layers.dense(z, n_units, activation=None,name='G_dense_1')
        H_layer1=tf.maximum(H_layer1, alpha*H_layer1)
        logits=tf.layers.dense(H_layer1, Output_dimension, activation=None,name='G_dense_2')
        Out=tf.nn.tanh(logits)
        return Out, logits

In [104]:
#def discriminator(x, n_units, reuse, alpha):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-104-600d0666813d>, line 1)

In [92]:
Output_dimension=724
n_units=128
reuse=False
alpha=0.01
tf.reset_default_graph() 
z=tf.placeholder(tf.float32, (None, 100), name="z")
a=generator(z,Output_dimension,n_units,reuse,alpha)

In [93]:
t_vars = tf.trainable_variables()

In [94]:
t_vars

[<tf.Variable 'generator/G_dense_1/kernel:0' shape=(100, 128) dtype=float32_ref>,
 <tf.Variable 'generator/G_dense_1/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'generator/G_dense_2/kernel:0' shape=(128, 724) dtype=float32_ref>,
 <tf.Variable 'generator/G_dense_2/bias:0' shape=(724,) dtype=float32_ref>]

In [99]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    Z= np.random.uniform(-1, 1, size=(16, 100))
    out,logits=sess.run(a,feed_dict={z:Z})
    bias=sess.run('generator/G_dense_1/kernel:0')

array([[-0.0663031 , -0.03332643,  0.07886194, ..., -0.00623073,
        -0.01737477,  0.03167973],
       [ 0.07791319,  0.13030823, -0.07311722, ...,  0.00733536,
         0.08427407, -0.01030193],
       [-0.07275196,  0.01118508,  0.01488778, ..., -0.02800812,
         0.03693193, -0.04016035],
       ...,
       [-0.00535052,  0.07286268, -0.10645548, ...,  0.13943537,
         0.07286045,  0.11181818],
       [-0.05919825,  0.13999136, -0.14724305, ...,  0.08042555,
         0.15920494, -0.153411  ],
       [-0.00241961,  0.12215142, -0.0614167 , ..., -0.11761044,
        -0.04104921, -0.12170555]], dtype=float32)